In [22]:
%reset -sf

In [74]:
import itertools
import random

n = 3
for comb in itertools.product(list(range(-10,10)), repeat=2):
    comb = [comb[0]] + [comb[1]]*(2*n-1)
    for _ in range(100):
        random.shuffle(comb)
        vsum = 0
        vprod = 1
        for x in comb[:n]:
            vsum += x
        for x in comb[n:]:
            vprod *= x
        if vprod != vsum:
            break
    else:
        print(comb, vprod, vsum)

[0, 0, 0, 0, 0, 0] 0 0


In [63]:
comb

[9, 9, 9, 9, 9, 9, 9, 9]

In [77]:
import itertools
import random

for comb in itertools.product(list(range(-7,8)), repeat=6):
    comb = list(comb)
    if comb == [0, 0, 0, 0, 0, 0]:
        continue
    for _ in range(100):
        random.shuffle(comb)
        a,b,c,d,e,f = comb
        if a+b+c != d*e*f:
            break
    else:
        break

In [78]:
comb

[6, 6, 6, 6, 6, 6]

In [76]:
import itertools
import random

for comb in itertools.product(list(range(-7,7)), repeat=4):
    comb = list(comb)
#     if comb == [0, 0, 0, 0] or comb == [2, 2, 2, 2]:
#         continue
    for _ in range(100):
        random.shuffle(comb)
        a,b,c,d = comb
        if a+b != c*d:
            break
    else:
        print(comb)
#         break

[2, -1, -1, -1]
[-1, 2, -1, -1]
[-1, 2, -1, -1]
[0, 0, 0, 0]
[2, -1, -1, -1]
[2, 2, 2, 2]


In [38]:
comb

[6, 6, 6, 6]